In [1]:
import os
import glob
import optuna
import warnings
import h5py as h5
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

from tqdm.auto import tqdm
from copy import deepcopy

from gensit.config import Config
from gensit.inputs import Inputs
from gensit.outputs import Outputs
from gensit.utils.misc_utils import *
from gensit.utils.math_utils import *
from gensit.utils.probability_utils import *
from gensit.contingency_table import instantiate_ct
from gensit.contingency_table.MarkovBasis import instantiate_markov_basis

In [2]:
%matplotlib inline

# AUTO RELOAD EXTERNAL MODULES
%load_ext autoreload
%autoreload 2

## Import samples

In [105]:
# Specify experiment id
experiment_id = "JointTableSIM_NN_SweepedNoise__totally_and_cell_constrained_21_05_2024_13_25_40"
# Specify experiment group id
experiment_group_id = 'exp1/'
dataset = 'DC'
experiment_dir = f'../../data/outputs/{dataset}/{experiment_group_id}/{experiment_id}/'
relative_experiment_dir = os.path.relpath(experiment_dir,os.getcwd())

In [106]:
# Create new logging object
logger = setup_logger(
    __name__,
    console_level = 'INFO',
    file_level = 'EMPTY'
)

In [107]:
# Output processing settings
settings = {
    "logging_mode": "INFO",
    "coordinate_slice": [
        "da.loss_name==str(['dest_attraction_ts_likelihood_loss', 'table_likelihood_loss'])",
        "da.sigma==0.14142"
    ],
    "slice":True,
    "metadata_keys":[],
    "burnin_thinning_trimming": [{'iter': {"burnin":10000, "thinning":1, "trimming":100000}}],
    "sample":["table"],
    "group_by":[],
    "filename_ending":"test",
    "force_reload":True,
    "n_workers": 1
}

In [ ]:
# Initialise outputs
jointgensit_outputs = Outputs(
    config = os.path.join(relative_experiment_dir,"config.json"),
    settings = settings,
    inputs = None,
    slice = True,
    level = 'NOTE'
)
# Silence outputs
jointgensit_outputs.logger.setLevels(console_level='NOTE')
# Load all data
jointgensit_outputs.load()

# Get data from first sweep of the SIM_NN experiment
jointgensit_outputs = jointgensit_outputs.get(0)

12:13.569 config INFO ----------------------------------------------------------------------------------
12:13.586 config INFO Parameter space size: 
 --- sigma: ['sigma', 'to_learn'] (3)
 --- loss_name: ['loss_name', 'loss_function', 'loss_kwargs'] (2)
12:13.603 config INFO Total = 6.
12:13.620 config INFO ----------------------------------------------------------------------------------
12:13.691 outputs INFO //////////////////////////////////////////////////////////////////////////////////
12:13.708 outputs INFO Slicing coordinates:
12:13.725 outputs INFO loss_name==str(['dest_attraction_ts_likelihood_loss', 'table_likelihood_loss'])
12:13.741 outputs INFO sigma==0.14142
12:13.758 outputs INFO iter: burnin = 10000, thinning = 1, trimming = 100000
12:13.775 outputs INFO //////////////////////////////////////////////////////////////////////////////////
12:13.792 outputs WARNING Removing ../../data/outputs/DC/exp1/JointTableSIM_NN_SweepedNoise__totally_and_cell_constrained_21_05_2024_1

In [ ]:
jointgensit_outputs.coords

In [19]:
inputs = Inputs(
    config = jointgensit_outputs.config,
    synthetic_data = False,
    logger = jointgensit_outputs.logger
)
inputs.cast_to_xarray()

56:37.241 inputs NOTE Loading Harris Wilson data ...
56:37.344 inputs NOTE Margins not provided
56:37.360 inputs NOTE Cells subset values file not provided


In [88]:
ground_truth_colsums = inputs.data.ground_truth_table.where(inputs.data.test_cells_mask,drop=True).sum('origin',skipna=True)

In [96]:
jointgensit_table_colsums_mean = jointgensit_outputs.data.table.mean('iter',dtype='float64',skipna=True).where(inputs.data.test_cells_mask,drop=True).sum('origin',skipna=True)

In [99]:
jointgensit_table_colsums_mean.sizes

Frozen({'destination': 43, 'sweep': 1})

In [101]:
ground_truth_colsums.sizes

Frozen({'destination': 43})

In [102]:
jointgensit_table_colsums_mean,ground_truth_colsums = xr.broadcast(jointgensit_table_colsums_mean,ground_truth_colsums)
jointgensit_table_colsums_mean,ground_truth_colsums = xr.align(jointgensit_table_colsums_mean,ground_truth_colsums, join='exact')
jointgensit_relative_colsum_l1_error = (jointgensit_table_colsums_mean-ground_truth_colsums)/ground_truth_colsums

In [103]:
abs(jointgensit_relative_colsum_l1_error).sum()

<xarray.DataArray ()>
array(223.16690963)

In [104]:
jointgensit_relative_colsum_l1_error.squeeze('sweep')

<xarray.DataArray (destination: 43)>
array([ 1.51445   ,  7.17402857,  3.63681729,  2.55589404,  5.4895352 ,
        3.11431184,  1.09844861,  6.08552976,  1.63587262,  2.37469561,
       12.7425    ,  5.91757667,  6.06300746,  4.61425694,  3.78155556,
        3.47162201, 10.79641842, 11.93563636,  6.78290615, 16.17286   ,
        6.80054894,  6.14509667,  3.41168346,  9.94626667,  2.8007432 ,
        6.52543786,  6.88512857,  8.39134091,  9.86309091,  9.618465  ,
       17.89713333,  1.35055375, -0.8277597 , -0.56051622,  4.79757177,
        0.32867054,  1.1864419 , -0.67729648, -0.92852449, -0.54554589,
        0.83753895,  5.79823086, -0.08540043])
Coordinates:
  * destination    (destination) int16 137 138 139 140 141 ... 176 177 178 179
    sweep          object ('none', "['alpha', 'beta', 'sigma']", "['dest_attr...
    sigma          <U4 'none'
    to_learn       <U26 "['alpha', 'beta', 'sigma']"
    loss_name      <U63 "['dest_attraction_ts_likelihood_loss', 'table_likeli...
    loss_function  <U20 "['custom', 'custom']"
    loss_kwargs    <U26 "{'noise_percentage': None}"

In [84]:
srmse(
    prediction = jointgensit_outputs.data.table.mean('iter',dtype='float64'),
    ground_truth = inputs.data.ground_truth_table,
    mask = inputs.data.test_cells_mask
)

<xarray.DataArray (sweep: 1)>
array([2.37211064])
Coordinates:
  * sweep          (sweep) object MultiIndex
  * sigma          (sweep) object 'none'
  * to_learn       (sweep) object "['alpha', 'beta', 'sigma']"
  * loss_name      (sweep) object "['dest_attraction_ts_likelihood_loss', 'ta...
  * loss_function  (sweep) object "['custom', 'custom']"
  * loss_kwargs    (sweep) object "{'noise_percentage': None}"

In [66]:
# Output processing settings
gmel_settings = {
    "logging_mode": "INFO",
    "coordinate_slice": [],
    "slice":False,
    "metadata_keys":[],
    "burnin_thinning_trimming": [],
    "sample":["intensity"],
    "group_by":[],
    "filename_ending":"test",
    "force_reload":True,
    "n_workers": 1
}

# Initialise outputs
gmel_outputs = Outputs(
    config = f'../../data/outputs/{dataset}/comparisons/GraphAttentionNetwork_Comparison_UnsetNoise__doubly_and_cell_constrained_all_region_features_16_05_2024_21_06_14/config.json',
    settings = gmel_settings,
    inputs = None,
    slice = True,
    level = 'NOTE'
)
# Silence outputs
gmel_outputs.logger.setLevels(console_level='NOTE')
# Load all data
gmel_outputs.load()

# Get data from first sweep of the experiment
gmel_outputs = gmel_outputs.get(0)

01:10.487 config INFO ----------------------------------------------------------------------------------
01:10.503 config INFO Parameter space size: 
 --- seed (10)
01:10.519 config INFO Total = 10.
01:10.535 config INFO ----------------------------------------------------------------------------------
01:10.551 outputs WARNING Removing ../../data/outputs/DC/comparisons/GraphAttentionNetwork_Comparison_UnsetNoise__doubly_and_cell_constrained_all_region_features_16_05_2024_21_06_14/sample_collections
01:10.719 outputs INFO Collecting samples intensity.
01:10.883 inputs NOTE Margins not provided
01:10.899 inputs NOTE Cells subset values file not provided
01:10.967 outputs NOTE Loading h5 data into xarrays...
01:13.802 outputs NOTE Populating data dictionary
01:14.233 outputs PROGRESS intensity: (1, 10000, 179, 179)
01:14.387 inputs NOTE Margins not provided
01:14.396 inputs NOTE Cells subset values file not provided
01:14.435 outputs NOTE Loading h5 data into xarrays...
01:16.366 outputs

In [67]:
gmel_intensity_colsums_mean = gmel_outputs.data.intensity.where(inputs.data.test_cells_mask,drop=True).groupby('seed').sum('origin').mean('iter',dtype='float64')

In [68]:
gmel_relative_colsum_l1_error = ((gmel_intensity_colsums_mean-ground_truth_colsums)/ground_truth_colsums).mean('seed',dtype='float64')

In [69]:
abs(gmel_relative_colsum_l1_error).sum()

<xarray.DataArray ()>
array(17.60251857)

In [77]:
gmel_outputs.data.intensity.groupby('seed').mean('iter',dtype='float64').groupby('seed').map(
    srmse,
    ground_truth = inputs.data.ground_truth_table,
    mask = inputs.data.test_cells_mask
).mean('seed')

<xarray.DataArray ()>
array(2.43013109)